---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [1]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [2]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [9]:
def answer_one():
    return df['Gold'].idxmax()

In [10]:
answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [11]:
def answer_two():
    df['diff'] = df['Gold'] - df['Gold.1']
    return df['diff'].idxmax()

In [12]:
answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [22]:
def answer_three():
    df_copy = df[(df['Gold'] >= 1) & (df['Gold.1'] >=1)]
    df_copy['relative_gold'] = (df['Gold'] - df['Gold.1']) / df['Gold.2']
    return df_copy['relative_gold'].idxmax()

In [23]:
answer_three()

/Users/dapenghan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [28]:
def answer_four():
    return df['Gold.2'] * 3 + df['Silver.2'] * 2 + df['Bronze.2'] * 1

In [29]:
answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

# Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [30]:
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [54]:
def answer_five():
    df_copy = census_df[census_df['SUMLEV'] == 50]
    df_copy = df_copy.groupby('STNAME').count()['SUMLEV']
    return df_copy.idxmax()

In [60]:
df_copy = census_df[census_df['SUMLEV'] == 50]
df_copy.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


In [61]:
df_copy = df_copy.groupby('STNAME')
df_copy.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411
69,50,4,9,2,13,Alaska,Aleutians East Borough,3141,3141,3169,...,-5.604858,-12.221204,-17.257039,-27.748454,-16.221087,19.305620,8.249313,5.752346,-3.920977,7.509763
70,50,4,9,2,16,Alaska,Aleutians West Census Area,5561,5561,5546,...,-11.496318,0.000000,-13.711875,-11.179039,-26.717891,1.257410,13.314397,0.175793,4.890830,-10.826858
71,50,4,9,2,20,Alaska,Anchorage Municipality,291826,291826,293405,...,-3.788123,-7.563712,-3.962343,-18.897015,-19.781254,-0.969922,-2.652342,0.053320,-14.789980,-15.280810
72,50,4,9,2,50,Alaska,Bethel Census Area,17013,17013,17080,...,0.057906,-3.926031,-9.731949,-13.525220,-14.441440,0.173717,-3.300142,-9.056901,-12.798659,-13.716580
73,50,4,9,2,60,Alaska,Bristol Bay Borough,997,997,1002,...,23.529412,-54.401583,-33.023736,-14.736842,-55.495103,23.529412,-54.401583,-33.023736,-14.736842,-55.495103


In [57]:
answer_five()

'Texas'

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [68]:
df_copy = census_df[census_df['SUMLEV'] == 50]
df_copy = df_copy.sort_values(by=['STNAME','CENSUS2010POP'],ascending=False).groupby('STNAME').head(3)    #df_copy = df_copy.set_index('diffRelative')
df_copy = df_copy.groupby('STNAME').sum().sort_values(by='CENSUS2010POP',ascending=False).head(3)#.index.tolist()

df_copy.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
STNAME,,,,,,,,,,,,,,,,,,,,,
California,150,12,27,18,169,15924150,15924274,15948261,16094942,16244137,...,-2.651916,-3.217407,-5.070710,-7.111949,-12.065453,10.408013,10.545720,9.005753,8.543736,3.920882
Texas,150,9,21,144,753,8269632,8270250,8298179,8439027,8601286,...,7.142972,14.863483,7.496872,9.559581,8.836789,20.398518,27.431740,21.195186,25.187793,24.341074
Illinois,150,6,9,51,171,6815061,6815323,6821742,6841179,6862181,...,-19.864534,-18.494617,-14.252015,-22.842075,-26.515534,-11.715929,-9.144169,-4.775492,-12.003940,-15.246911


In [71]:
def answer_six():
    df_copy = census_df[census_df['SUMLEV'] == 50]
    df_copy = df_copy.sort_values(by=['STNAME','CENSUS2010POP'],ascending=False).groupby('STNAME').head(3)    #df_copy = df_copy.set_index('diffRelative')
    df_copy = df_copy.groupby('STNAME').sum().sort_values(by='CENSUS2010POP',ascending=False).head(3).index.tolist()

    return df_copy

In [72]:
answer_six()

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [171]:
df_copy = census_df[census_df['SUMLEV'] == 50]

#df_copy['population_change'] = df_copy[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].max(axis =1)
#-df_copy[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].min(axis=1)
df_copy['population_change'] = df_copy[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].max(axis =1)-df_copy[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].min(axis=1)
# FIXME, above two are not equal ???? need to find out about python change line
df_copy.head()


/Users/dapenghan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,population_change
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333,687
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499,20516
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299,852
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861,349
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411,403


In [179]:
    df_copy = census_df[census_df['SUMLEV'] == 50]
    df_population = df_copy[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']]
    df_pop_max = df_population.max(axis = 1)
    df_pop_min = df_population.min(axis = 1)
    df_copy['population_change'] = df_pop_max - df_pop_min
    df_copy.sort_values(by='population_change', ascending=False)

/Users/dapenghan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,population_change
2667,50,3,7,48,201,Texas,Harris County,4092459,4093076,4108187,...,3.518819,4.820913,5.243613,3.794291,7.154417,8.915715,10.683269,11.850629,10.303814,429841
209,50,4,9,6,37,California,Los Angeles County,9818605,9818700,9826009,...,-4.071266,-4.332318,-5.394299,-6.027694,-0.007808,0.377510,0.578049,0.215038,-0.437481,344283
106,50,4,8,4,13,Arizona,Maricopa County,3817117,3817357,3825597,...,9.232973,7.879873,8.318210,9.123309,4.726436,11.527593,10.390177,11.203958,11.985998,342350
227,50,4,9,6,73,California,San Diego County,3095313,3095308,3104386,...,0.235513,-0.246567,0.657077,-2.855045,4.382515,5.773072,4.745940,5.921091,2.770965,195135
372,50,3,5,12,86,Florida,Miami-Dade County,2496435,2498017,2508171,...,-6.908149,-8.130040,-10.464276,-12.205475,23.110902,6.891198,6.633403,6.087936,4.154777,184946
2623,50,3,7,48,113,Texas,Dallas County,2368139,2367643,2373464,...,4.853046,-2.117265,-0.754784,-0.981666,5.233271,9.100284,2.574144,4.639783,4.397784,179921
3017,50,4,9,53,33,Washington,King County,1931249,1931256,1937699,...,5.472471,5.662073,3.337122,2.840873,11.616146,12.013317,12.704538,11.206066,10.589099,179426
2581,50,3,7,48,29,Texas,Bexar County,1714773,1714774,1722989,...,7.531302,7.395983,8.624810,8.111171,10.352153,10.828220,10.370317,11.818602,11.417693,174764
2786,50,3,7,48,439,Texas,Tarrant County,1809034,1809531,1816528,...,6.491618,4.793224,5.070751,6.024164,8.010830,9.415741,7.937773,8.697380,9.639477,165970
1777,50,4,8,32,3,Nevada,Clark County,1951269,1951269,1953378,...,6.206036,5.494924,9.848283,11.903114,0.531032,9.344107,8.803899,13.633456,15.680887,161423


In [175]:
df_copy['population_change'].idxmax()

2667

In [183]:
df_copy.loc[df_copy['population_change'].idxmax()]['CTYNAME'] # should use loc not .iloc, index is not number index

'Harris County'

In [177]:
df_copy.sort_values(by='population_change')[::-1]['CTYNAME'].iloc[0]


'Harris County'

In [184]:
def answer_seven():
    df_copy = census_df[census_df['SUMLEV'] == 50]
    df_copy['population_change'] = df_copy[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].max(axis =1)- df_copy[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].min(axis=1)
    return df_copy.loc[df_copy['population_change'].idxmax()]['CTYNAME']

In [185]:
answer_seven()

/Users/dapenghan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [131]:
def answer_eight():
    df_copy = census_df[census_df['SUMLEV'] == 50]
    df_copy = df_copy[((df_copy['REGION'] == 1) | (df_copy['REGION'] == 2)) 
                      & (df_copy['CTYNAME'].str.startswith('Washington'))
                      & (df_copy['POPESTIMATE2015'] > df_copy['POPESTIMATE2014'])]

    return df_copy[['STNAME', 'CTYNAME']]

In [132]:
answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
